In [1]:
# Import needed libraries
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
# Filepaths
mapdata_fp = "data/maakunnat_2021_milj.shp"
info_fp = "data/info.txt"

# Read files
mapdata = gpd.read_file(mapdata_fp)
info = pd.read_csv(info_fp, sep=';')

In [3]:
# Merge data sets
data = mapdata.merge(info, left_on='NAMEFIN', right_on='name')

In [ ]:
# Plot figure
fig, ax = plt.subplots(figsize=(5,10))

data.plot(ax=ax, 
          column="pop_den", 
          cmap="OrRd",
          scheme="natural_breaks",
          k=8, 
          legend=True,
          )

# Set legend
ax.get_legend().set_bbox_to_anchor((1.14,1))
ax.get_legend().set_title("Population density \n(per square kilometers)")

# Save figure
output_fp = 'docs/staticmap'
plt.savefig(output_fp)